In [1]:
from pyspark import SparkFiles

In [2]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df =  spark.read.option('header', 'true').csv("file://"+SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
 US| 2975964|R1NBG94582SJE2|B00I01JQJM| 860486164|GoPro Rechargeabl...| Camera| 5| 0| 0| N| Y| Five Stars| ok|2015-08-31 00:00:00|
 US| 23526356|R273DCA6Y0H9V7|B00TCO0ZAA| 292641483|Professional 58mm...| Camera| 5| 0| 0| N| Y| Love it!!!|Perfect, even stu...|2015-08-31 00:00:00|
 US| 52764145| RQVOXO7WUOFK6|B00B7733E0| 75825744|Spy Tec Z12 Motio...| Camera| 2| 1| 1| N| Y|Another Motion De...|If the words, &#3...|2015-08-31 00:00:00|
 US| 47348933|R1KWKSF21PO6HO|B006ZN4U34| 789352955|Celestron UpClose...| Camera| 5| 0| 0| N| Y|Exactly what I wa...|Exactly what I wa...|2015-08-31 00:00:00|
 US| 33680700|R38H3UO1J190GI|B00HUEBGMU| 19067902|Vidpro XM-L Wired...| Camera| 5| 1| 1| N| Y|Good mic at a Goo...|I will look past ...|2015-08-31 00:00:00|
 US| 30301059|R3NPIFKLR19NQA|B008MW6Y12| 597683407|NIX 8 inch Hi-Res...| Camera| 3| 0| 0| N| Y|The controls are ...|The controls are ...|2015-08-31 00:00:00|
 US| 28282645|R3MBE6UCH3435E|B00TE8XKIS| 35563334|Polaroid ZIP Mobi...| Camera| 3| 8| 8| N| N|Fun toy for makin...|The printer came ...|2015-08-31 00:00:00|
 US| 502818|R2E7A4FF0PVY5Q|B00ZKDUFBQ| 555190742|GeekPro 2.0-Inch ...| Camera| 5| 0| 1| N| Y| Five Stars|Great camera for ...|2015-08-31 00:00:00|
 US| 1481233| R3R8JDQ2BF4NM|B010BZ7S2Q| 129544315|Sony HDR-AZ1VR Ac...| Camera| 5| 0| 2| N| Y| Five Stars|Product is very g...|2015-08-31 00:00:00|
 US| 27885926|R1YND4BS823GN5|B00HRXSSRA| 708418657|ChiliPower DMW-BL...| Camera| 1| 0| 0| N| Y| Sucky.|Lasted a few hour...|2015-08-31 00:00:00|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
only showing top 10 rows

In [3]:
# Count 
df.count()

Out[59]: 1801974

In [4]:
#
df_select = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
df_select.show()

+--------------+-----------+-------------+-----------+----+
 review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
R1NBG94582SJE2| 5| 0| 0| N|
R273DCA6Y0H9V7| 5| 0| 0| N|
 RQVOXO7WUOFK6| 2| 1| 1| N|
R1KWKSF21PO6HO| 5| 0| 0| N|
R38H3UO1J190GI| 5| 1| 1| N|
R3NPIFKLR19NQA| 3| 0| 0| N|
R3MBE6UCH3435E| 3| 8| 8| N|
R2E7A4FF0PVY5Q| 5| 0| 1| N|
 R3R8JDQ2BF4NM| 5| 0| 2| N|
R1YND4BS823GN5| 1| 0| 0| N|
R2TZNSA18V7YF6| 4| 1| 1| N|
R22ZVRDPPXIDNL| 5| 0| 0| N|
R1F4O6WOO2W461| 3| 3| 3| N|
 RT1KLS3QSJNUT| 5| 4| 4| N|
R222VYJL5K5IRS| 5| 0| 0| N|
R31LUR7M4PQOLU| 5| 0| 0| N|
R1OD3T3Q042LUQ| 2| 0| 0| N|
R2QT68OZTT2YKE| 5| 0| 0| N|
 RMFQF59FG3TD1| 4| 0| 0| N|
R1QNYFW6G31R5T| 1| 1| 3| N|
+--------------+-----------+-------------+-----------+----+
only showing top 20 rows

In [5]:
df_vote = df_select.filter(df_select['total_votes'] >= 10)
df_vote_select = df_vote.filter(df_vote["helpful_votes"]/df_vote["total_votes"] >= 0.5)

In [6]:
vine_df = df_vote_select.filter(df_vote_select['vine']== 'Y')
nvine_df = df_vote_select.filter(df_vote_select['vine']== 'N')

vine_df.describe().show()
nvine_df.describe().show()

+-------+--------------+------------------+------------------+------------------+----+
summary| review_id| star_rating| helpful_votes| total_votes|vine|
+-------+--------------+------------------+------------------+------------------+----+
 count| 1171| 1171| 1171| 1171|1171|
 mean| null| 4.140051238257899| 48.04526046114432| 52.4064901793339|null|
 stddev| null|0.9612276680636224|109.71629537633561|113.66609711367433|null|
 min|R102N6L9WI2ZC8| 1| 5| 10| Y|
 max| RZX9XBNZSRBS| 5| 2206| 2245| Y|
+-------+--------------+------------------+------------------+------------------+----+

+-------+--------------+------------------+-----------------+------------------+------+
summary| review_id| star_rating| helpful_votes| total_votes| vine|
+-------+--------------+------------------+-----------------+------------------+------+
 count| 107112| 107112| 107112| 107112|107112|
 mean| null|3.8304204944357307|34.72531555754724|38.135157592053176| null|
 stddev| null| 1.459788804046264|87.70895417279084| 91.01977159424837| null|
 min|R10051RMU125BG| 1| 5| 10| N|
 max| RZZYNOOZS8QAY| 5| 5132| 5287| N|
+-------+--------------+------------------+-----------------+------------------+------+

In [7]:
from pyspark.sql.functions import col, avg
# number of five star reviews
vine_five_star_number = vine_df.select("star_rating").filter(vine_df['star_rating'] == 5).count()
# number of reviews
vine_number = vine_df.count()

print(vine_number)
print(vine_five_star_number)
# average rating
vine_average_rating = vine_df.agg(avg(col("star_rating"))).show()
# number of helpful votes
vine_helpful_votes = vine_df.select("helpful_votes").groupBy().sum().show()
# Percent five star rating
percentage_five_star_vine = vine_five_star_number / vine_number
print(percentage_five_star_vine)

1171
518
+-----------------+
 avg(star_rating)|
+-----------------+
4.140051238257899|
+-----------------+

+------------------+
sum(helpful_votes)|
+------------------+
 56261|
+------------------+

0.44235695986336465

In [8]:
# number of five star reviews
nvine_five_star_number = nvine_df.select("star_rating").filter(nvine_df['star_rating'] == 5).count()
# number of reviews
nvine_number = nvine_df.count()

print(nvine_number)
print(nvine_five_star_number)
# average rating
nvine_average_rating = nvine_df.agg(avg(col("star_rating"))).show()
# number of helpful votes
nvine_helpful_votes = nvine_df.select("helpful_votes").groupBy().sum().show()
# Percent five star rating
percentage_five_star_nvine = nvine_five_star_number / nvine_number
print(percentage_five_star_nvine)

107112
52960
+------------------+
 avg(star_rating)|
+------------------+
3.8304204944357307|
+------------------+

+------------------+
sum(helpful_votes)|
+------------------+
 3719498|
+------------------+

0.4944357308238106